In [54]:
from google.colab import files
uploaded = files.upload()

Saving 1unb_multi.csv to 1unb_multi (2).csv


In [55]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Load the dataset
url_dataset = pd.read_csv('1unb_multi.csv')  # Replace 'url_dataset.csv' with your actual dataset file name

# Separate the features and labels
X = url_dataset.iloc[:, 2:]  # Select all columns except the last one
y = url_dataset['Label']  # Select the last column as the target variable

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Train the models
rf_model = RandomForestClassifier(n_estimators=200, criterion='gini', min_samples_split=2, max_depth=100, random_state=42)
rf_model.fit(X_train, y_train)


dt_model = DecisionTreeClassifier(max_depth=100, criterion='gini', splitter='best')
dt_model.fit(X_train, y_train)

# Create the neural network model
model = Sequential()
model.add(Dense(20, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, pd.get_dummies(y_train), epochs=500, batch_size=512, verbose=1)



Epoch 1/500
227/227 [==============================] - 1s 2ms/step - loss: 1.0248 - accuracy: 0.6683
Epoch 2/500
227/227 [==============================] - 0s 2ms/step - loss: 0.6887 - accuracy: 0.7743
Epoch 3/500
227/227 [==============================] - 0s 2ms/step - loss: 0.5999 - accuracy: 0.7955
Epoch 4/500
227/227 [==============================] - 0s 2ms/step - loss: 0.5580 - accuracy: 0.8069
Epoch 5/500
227/227 [==============================] - 0s 2ms/step - loss: 0.5323 - accuracy: 0.8145
Epoch 6/500
227/227 [==============================] - 1s 3ms/step - loss: 0.5128 - accuracy: 0.8225
Epoch 7/500
227/227 [==============================] - 1s 2ms/step - loss: 0.4975 - accuracy: 0.8297
Epoch 8/500
227/227 [==============================] - 1s 3ms/step - loss: 0.4834 - accuracy: 0.8345
Epoch 9/500
227/227 [==============================] - 1s 2ms/step - loss: 0.4699 - accuracy: 0.8394
Epoch 10/500
227/227 [==============================] - 1s 2ms/step - loss: 0.4585 - accura

In [56]:
# Make predictions on the test data
rf_predictions = rf_model.predict(X_test)
dt_predictions = dt_model.predict(X_test)
nn_predictions = model.predict(X_test)
nn_predictions_classes = np.argmax(nn_predictions, axis=1)

# Create the ensemble prediction using majority voting
ensemble_predictions = []
for rf_pred, dt_pred, nn_pred in zip(rf_predictions, dt_predictions, nn_predictions_classes):
    votes = [rf_pred, dt_pred, nn_pred]
    majority_vote = max(set(votes), key=votes.count)
    ensemble_predictions.append(majority_vote)

# Decode the predictions
ensemble_predictions_decoded = label_encoder.inverse_transform(ensemble_predictions)

# Decode the true labels
y_test_decoded = label_encoder.inverse_transform(y_test)

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_test_decoded, ensemble_predictions_decoded)
print("Ensemble Accuracy:", ensemble_accuracy)

# Print classification report with 5 decimal places
ensemble_report = classification_report(y_test_decoded, ensemble_predictions_decoded, digits=5)
print("Ensemble Classification Report:")
print(ensemble_report)

1551/1551 [==============================] - 3s 2ms/step
Ensemble Accuracy: 0.9811529933481153
Ensemble Classification Report:
              precision    recall  f1-score   support

  Defacement    0.97864   0.99498   0.98674     29060
      benign    0.99068   0.99574   0.99320     10568
     malware    0.98883   0.97058   0.97962      3467
    phishing    0.94938   0.81325   0.87606      2929
        spam    0.98867   0.97351   0.98103      3586

    accuracy                        0.98115     49610
   macro avg    0.97924   0.94961   0.96333     49610
weighted avg    0.98092   0.98115   0.98067     49610

